
*   List item
*   List item



In [ ]:

from google.colab import drive
drive.mount('...')


# Load data and prepare it for training
file_path = ".../PD.xlsx"


Mounted at /content/drive


In [ ]:
# Import Libraries

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from imblearn.under_sampling import RandomUnderSampler
import random

In [ ]:
#Load data

In [ ]:
df = pd.read_excel(file_path)
features = 	['mdvp_fo_hz','mdvp_fhi_hz', 'mdvp_flo_hz',	'mdvp_jitter', 	'mdvp_jitter_abs', 'mdvp_rap', 'mdvp_ppq', 'jitter_ddp', 'mdvp_shimmer',	'mdvp_shimmer_db',	'shimmer_apq3',	'shimmer_apq5',	'mdvp_apq',	'shimmer_dda',	'nhr',	'hnr', 'rpde',	'dfa',	'spread1',	'spread2',	'd2',	'ppe'	]
X = df[features]
y = df['status']

In [ ]:
#Spilt data to train and test sets

In [ ]:
Num_Test = 10
Healthy_Index = []
Healthy_Index = df[df['status']==False].index
HI = Healthy_Index.to_list()
Test_Healthy_Index = random.sample(HI, Num_Test)
THI=Test_Healthy_Index
HI2= [x for x in HI if x not in THI]
Test_Healthy = df.loc[THI]
Train_Healthy = df.loc[HI2]
PD_Index = []
PD_Index = df[df['status']== True].index
PD_Index = PD_Index.to_list()
Test_PD_Index = random.sample(PD_Index, Num_Test)
TPD=Test_PD_Index
PD_Index2= [x for x in PD_Index if x not in TPD]
Test_PD = df.loc[TPD]
Train_PD = df.loc[PD_Index2]

In [ ]:
Testdata = pd.concat([Test_PD, Test_Healthy], ignore_index=True)

In [ ]:
import os
import pandas as pd
df = pd.DataFrame(Testdata)
# Specify the full path where you want to save the Excel file in Colab
folder_path = "../PD/"
file_name = 'PD_Test.xlsx'
os.makedirs(folder_path, exist_ok = True)
df.to_excel(os.path.join(folder_path, file_name), index=False)

# Print a message indicating the file has been saved
print(f"Excel file saved at: {folder_path}{file_name}")


Excel file saved at: /content/drive/MyDrive/CIHR/PD/PD_Test.xlsx


In [ ]:
Traindata = pd.concat([Train_PD,Train_Healthy], ignore_index=True)
len(data)

175

In [ ]:

df = pd.DataFrame(Traindata)
# Specify the full path where you want to save the Excel file in Colab
folder_path = "../PD/"
file_name = 'PD_Train.xlsx'
os.makedirs(folder_path, exist_ok = True)
df.to_excel(os.path.join(folder_path, file_name), index=False)

# Print a message indicating the file has been saved
print(f"Excel file saved at: {folder_path}{file_name}")

Excel file saved at: /content/drive/MyDrive/CIHR/PD/PD_Train.xlsx


In [ ]:
data = Traindata

In [ ]:
X = data[features]
y = data['status']
len(y)
class_counts = pd.Series(y).value_counts()
print(class_counts)

True     137
False     38
Name: status, dtype: int64


In [ ]:
! pip install sdv

In [ ]:
from sdv.single_table import CopulaGANSynthesizer
from sdv.metadata import SingleTableMetadata

real_data = []
real_data = X.join(y)
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=real_data)
synthesizer = CopulaGANSynthesizer(metadata)

synthesizer.fit(real_data)
synthetic_data = synthesizer.sample(num_rows=len(y))

In [ ]:
from sdv.evaluation.single_table import evaluate_quality

quality_report = evaluate_quality(
    real_data,
    synthetic_data,
    metadata)

Generating report ...
(2/2) Evaluating Column Pair Trends: : 100%|██████████| 253/253 [00:02<00:00, 94.64it/s]

Overall Score: 67.94%

Properties:
- Column Shapes: 62.21%
- Column Pair Trends: 73.67%


In [ ]:
New_data = real_data.append(synthetic_data, ignore_index=True)
#New_data = real_data.concat(synthetic_data, ignore_index=True)
New_data
len(New_data)

<ipython-input-123-fd1b6b317d92>:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  New_data = real_data.append(synthetic_data, ignore_index=True)


350

In [ ]:
df = pd.DataFrame(New_data)
# Specify the full path where you want to save the Excel file in Colab
folder_path = "../PD/"
file_name = 'PD_CopulaGAN.xlsx'
os.makedirs(folder_path, exist_ok = True)
df.to_excel(os.path.join(folder_path, file_name), index=False)

# Print a message indicating the file has been saved
print(f"Excel file saved at: {folder_path}{file_name}")

Excel file saved at: /content/drive/MyDrive/CIHR/PD/PD_CopulaGAN.xlsx


In [ ]:
X = New_data[features]
y =  New_data['status']
class_counts = pd.Series(y).value_counts()
print(class_counts)

True     208
False    142
Name: status, dtype: int64


In [ ]:
from imblearn.under_sampling import NearMiss
undersample = NearMiss(version=1, n_neighbors=3)
# transform the dataset
X_S, y_S = undersample.fit_resample(X, y)
class_counts = pd.Series(y_S).value_counts()
print(class_counts)
NearMiss1_Data= pd.concat([X_S, y_S], axis =1)

False    142
True     142
Name: status, dtype: int64


In [ ]:
df = pd.DataFrame(NearMiss1_Data)
# Specify the full path where you want to save the Excel file in Colab
folder_path = "../PD/"
file_name = 'PD_CoGAN_NearMiss1.xlsx'
os.makedirs(folder_path, exist_ok = True)
df.to_excel(os.path.join(folder_path, file_name), index=False)

# Print a message indicating the file has been saved
print(f"Excel file saved at: {folder_path}{file_name}")

Excel file saved at: /content/drive/MyDrive/CIHR/PD/PD_CoGAN_NearMiss1.xlsx


In [ ]:
from imblearn.under_sampling import NearMiss
undersample = NearMiss(version=2, n_neighbors=3)
# transform the dataset
X_S, y_S = undersample.fit_resample(X, y)
class_counts = pd.Series(y).value_counts()
print(class_counts)
NearMiss2_Data= pd.concat([X_S, y_S], axis =1)

True     208
False    142
Name: status, dtype: int64


In [ ]:
df = pd.DataFrame(NearMiss2_Data)
# Specify the full path where you want to save the Excel file in Colab
folder_path = "/content/drive/MyDrive/CIHR/PD/"
file_name = 'PD_CoGAN_NearMiss2.xlsx'
os.makedirs(folder_path, exist_ok = True)
df.to_excel(os.path.join(folder_path, file_name), index=False)

# Print a message indicating the file has been saved
print(f"Excel file saved at: {folder_path}{file_name}")

Excel file saved at: /content/drive/MyDrive/CIHR/PD/PD_CoGAN_NearMiss2.xlsx


In [ ]:
from imblearn.under_sampling import NearMiss
undersample = NearMiss(version=3, n_neighbors=3)
# transform the dataset
X_S, y_S = undersample.fit_resample(X, y)
class_counts = pd.Series(y_S).value_counts()
print(class_counts)
NearMiss3_Data= pd.concat([X_S, y_S], axis =1)

False    142
True     138
Name: status, dtype: int64


/usr/local/lib/python3.10/dist-packages/imblearn/under_sampling/_prototype_selection/_nearmiss.py:203: UserWarning: The number of the samples to be selected is larger than the number of samples available. The balancing ratio cannot be ensure and all samples will be returned.
  warnings.warn(


In [ ]:
df = pd.DataFrame(NearMiss3_Data)
# Specify the full path where you want to save the Excel file in Colab
folder_path = "../PD/"
file_name = 'PD_CoGAN_NearMiss3.xlsx'
os.makedirs(folder_path, exist_ok = True)
df.to_excel(os.path.join(folder_path, file_name), index=False)

# Print a message indicating the file has been saved
print(f"Excel file saved at: {folder_path}{file_name}")

Excel file saved at: /content/drive/MyDrive/CIHR/PD/PD_CoGAN_NearMiss3.xlsx


In [ ]:
from imblearn.under_sampling import CondensedNearestNeighbour
undersample = CondensedNearestNeighbour(n_neighbors=3)
X_S, y_S = undersample.fit_resample(X, y)
# transform the dataset
class_counts = pd.Series(y_S).value_counts()
print(class_counts)
CNN_Data= pd.concat([X_S, y_S], axis =1)

False    142
True      98
Name: status, dtype: int64


In [ ]:
df = pd.DataFrame(CNN_Data)
# Specify the full path where you want to save the Excel file in Colab
folder_path = "../PD/"
file_name = 'PD_CoGAN_CNN.xlsx'
os.makedirs(folder_path, exist_ok = True)
df.to_excel(os.path.join(folder_path, file_name), index=False)

# Print a message indicating the file has been saved
print(f"Excel file saved at: {folder_path}{file_name}")

Excel file saved at: /content/drive/MyDrive/CIHR/PD/PD_CoGAN_CNN.xlsx


In [ ]:
from imblearn.under_sampling import TomekLinks
undersample = TomekLinks(sampling_strategy='majority')
X_S, y_S = undersample.fit_resample(X, y)
# transform the dataset
class_counts = pd.Series(y_S).value_counts()
print(class_counts)
TomekLinks_Data= pd.concat([X_S, y_S], axis =1)

True     182
False    142
Name: status, dtype: int64


In [ ]:
df = pd.DataFrame(TomekLinks_Data)
# Specify the full path where you want to save the Excel file in Colab
folder_path = "/content/drive/MyDrive/CIHR/PD/"
file_name = 'PD_CoGAN_TomekLinks.xlsx'
os.makedirs(folder_path, exist_ok = True)
df.to_excel(os.path.join(folder_path, file_name), index=False)

# Print a message indicating the file has been saved
print(f"Excel file saved at: {folder_path}{file_name}")

Excel file saved at: /content/drive/MyDrive/CIHR/PD/PD_CoGAN_TomekLinks.xlsx


In [ ]:
from imblearn.under_sampling import EditedNearestNeighbours
undersample = EditedNearestNeighbours()
X_S, y_S = undersample.fit_resample(X, y)
# transform the dataset
class_counts = pd.Series(y_S).value_counts()
print(class_counts)
ENN_Data= pd.concat([X_S, y_S], axis =1)

False    142
True     109
Name: status, dtype: int64


In [ ]:
df = pd.DataFrame(ENN_Data)
# Specify the full path where you want to save the Excel file in Colab
folder_path = "../PD/"
file_name = 'PD_CoGAN_ENN.xlsx'
os.makedirs(folder_path, exist_ok = True)
df.to_excel(os.path.join(folder_path, file_name), index=False)

# Print a message indicating the file has been saved
print(f"Excel file saved at: {folder_path}{file_name}")

Excel file saved at: /content/drive/MyDrive/CIHR/PD/PD_CoGAN_ENN.xlsx


In [ ]:
from imblearn.under_sampling import ClusterCentroids
undersample = ClusterCentroids(random_state=0)
X_S, y_S = undersample.fit_resample(X, y)
class_counts = pd.Series(y_S).value_counts()
print(class_counts)
CC_Data= pd.concat([X_S, y_S], axis =1)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


False    142
True     142
Name: status, dtype: int64


In [ ]:
df = pd.DataFrame(CC_Data)
# Specify the full path where you want to save the Excel file in Colab
folder_path = "../PD/"
file_name = 'PD_CoGAN_CC.xlsx'
os.makedirs(folder_path, exist_ok = True)
df.to_excel(os.path.join(folder_path, file_name), index=False)

# Print a message indicating the file has been saved
print(f"Excel file saved at: {folder_path}{file_name}")

Excel file saved at: /content/drive/MyDrive/CIHR/PD/PD_CoGAN_CC.xlsx


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
undersample= RandomUnderSampler(random_state=0)
X_S, y_S = undersample.fit_resample(X, y)
class_counts = pd.Series(y_S).value_counts()
print(class_counts)
RUS_Data= pd.concat([X_S, y_S], axis =1)

False    142
True     142
Name: status, dtype: int64


In [ ]:
df = pd.DataFrame(RUS_Data)
# Specify the full path where you want to save the Excel file in Colab
folder_path = "../PD/"
file_name = 'PD_CoGAN_RUS.xlsx'
os.makedirs(folder_path, exist_ok = True)
df.to_excel(os.path.join(folder_path, file_name), index=False)

# Print a message indicating the file has been saved
print(f"Excel file saved at: {folder_path}{file_name}")

Excel file saved at: /content/drive/MyDrive/CIHR/PD/PD_CoGAN_RUS.xlsx


In [ ]:
from sklearn.linear_model import LogisticRegression
from imblearn.under_sampling import InstanceHardnessThreshold
undersample = InstanceHardnessThreshold(random_state=0, estimator=LogisticRegression(solver='lbfgs', multi_class='auto'))
X_S= []
y_S = []
X_S, y_S = undersample.fit_resample(X, y)
class_counts = pd.Series(y_S).value_counts()
print(class_counts)
IHT_Data= pd.concat([X_S, y_S], axis =1)

In [ ]:
from imblearn.under_sampling import RepeatedEditedNearestNeighbours
undersample = RepeatedEditedNearestNeighbours()
X_S, y_S = undersample.fit_resample(X, y)
class_counts = pd.Series(y_S).value_counts()
print(class_counts)
RENN_Data= pd.concat([X_S, y_S], axis =1)


False    142
True     109
Name: status, dtype: int64


In [ ]:
df = pd.DataFrame(RENN_Data)
# Specify the full path where you want to save the Excel file in Colab
folder_path = "../PD/"
file_name = 'PD_CoGAN_RENN.xlsx'
os.makedirs(folder_path, exist_ok = True)
df.to_excel(os.path.join(folder_path, file_name), index=False)

# Print a message indicating the file has been saved
print(f"Excel file saved at: {folder_path}{file_name}")

Excel file saved at: /content/drive/MyDrive/CIHR/PD/PD_CoGAN_RENN.xlsx


In [ ]:
from imblearn.under_sampling import AllKNN
undersample = AllKNN()
X_S, y_S = undersample.fit_resample(X, y)
class_counts = pd.Series(y_S).value_counts()
print(class_counts)
AllKNN_Data= pd.concat([X_S, y_S], axis =1)

False    142
True     122
Name: status, dtype: int64


In [ ]:
df = pd.DataFrame(AllKNN_Data)
# Specify the full path where you want to save the Excel file in Colab
folder_path = "../PD/"
file_name = 'PD_CoGAN_AllKNN.xlsx'
os.makedirs(folder_path, exist_ok = True)
df.to_excel(os.path.join(folder_path, file_name), index=False)
# Print a message indicating the file has been saved
print(f"Excel file saved at: {folder_path}{file_name}")

Excel file saved at: /content/drive/MyDrive/CIHR/PD/PD_CoGAN_AllKNN.xlsx


In [ ]:
from imblearn.under_sampling import OneSidedSelection
undersample = OneSidedSelection(n_neighbors=1, n_seeds_S=200)
# transform the dataset
X_S, y_S = undersample.fit_resample(X, y)
class_counts = pd.Series(y_S).value_counts()
print(class_counts)
OSS_Data= pd.concat([X_S, y_S], axis =1)

True     204
False    142
Name: status, dtype: int64


In [ ]:
df = pd.DataFrame(OSS_Data)
# Specify the full path where you want to save the Excel file in Colab
folder_path = "../PD/"
file_name = 'PD_CoGAN_OSS_Data.xlsx'
os.makedirs(folder_path, exist_ok = True)
df.to_excel(os.path.join(folder_path, file_name), index=False)
# Print a message indicating the file has been saved
print(f"Excel file saved at: {folder_path}{file_name}")

Excel file saved at: /content/drive/MyDrive/CIHR/PD/PD_CoGAN_OSS_Data.xlsx
